In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

## Спарсим данные

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=менеджер&no_magic=true&L_save_area=true&items_on_page=20'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [5]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page

Cтраница №32, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=31&hhtmFrom=vacancy_search_list
Cтраница №33, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=32&hhtmFrom=vacancy_search_list
Cтраница №34, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=33&hhtmFrom=vacancy_search_list
Cтраница №35, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

Cтраница №63, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=62&hhtmFrom=vacancy_search_list
Cтраница №64, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=63&hhtmFrom=vacancy_search_list
Cтраница №65, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=64&hhtmFrom=vacancy_search_list
Cтраница №66, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

Cтраница №94, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=93&hhtmFrom=vacancy_search_list
Cтраница №95, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=94&hhtmFrom=vacancy_search_list
Cтраница №96, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&no_magic=true&L_save_area=true&items_on_page=20&page=95&hhtmFrom=vacancy_search_list
Cтраница №97, ссылка: https://spb.hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80&n

In [6]:
len(result)

2000

## Установим `mongo` сервер и клиент

In [27]:
!apt install mongodb > log

"apt" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [28]:
!service mongodb start


"service" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [29]:
!pip install pymongo

## Запишем данные в базу

In [30]:
from pymongo import MongoClient

In [31]:
client = MongoClient()

In [32]:
db = client.vacancies_manager_hh

In [33]:
collection_vacancies_hh_ru = db.hh_ru

In [34]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [35]:
cheak_and_save_vacancies_in_db(result)

In [36]:
result_find = list(collection_vacancies_hh_ru.find())

In [37]:
len(result_find)

1914

In [38]:
pprint(result)

[{'_id': ObjectId('64046b42d06070ad98168f26'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 100000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77555675?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам (Малоэтажное строительство)',
  'vacancy_salary': 'от 100\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168f27'),
  'currency_salary': 'руб.',
  'max_salary': 85000,
  'min_salary': 50000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/70737330?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам/Старший менеджер по продажам',
  'vacancy_salary': '50\u202f000 – 85\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168f28'),
  'currency_salary': 'руб.',
  'max_salary': 110000,
  'min_salary': 30000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77542065?from=vac

 {'_id': ObjectId('64046b42d06070ad98168f83'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 150000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77069242?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам (г. Казань)',
  'vacancy_salary': 'от 150\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168f84'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 60000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/75527778?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам',
  'vacancy_salary': 'от 60\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168f85'),
  'currency_salary': 'руб.',
  'max_salary': 80000,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76966792?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6

  'vacancy_salary': '35\u202f000 – 60\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168fbe'),
  'currency_salary': 'руб.',
  'max_salary': 100000,
  'min_salary': 45000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77179952?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам, удаленно',
  'vacancy_salary': '45\u202f000 – 100\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168fbf'),
  'currency_salary': 'руб.',
  'max_salary': 120000,
  'min_salary': 55000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76968474?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам',
  'vacancy_salary': '55\u202f000 – 120\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168fc0'),
  'currency_salary': 'руб.',
  'max_salary': 42000,
  'min_salary': 35000,
  'vacancy_l

  'max_salary': 150000,
  'min_salary': 60000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77300317?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам (Digital, удаленно)',
  'vacancy_salary': '60\u202f000 – 150\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168ff9'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 90000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/72016123?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Региональный менеджер /Менеджер по развитию продаж B2B '
                  '(Казань, светотехника) удаленно',
  'vacancy_salary': 'от 90\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b42d06070ad98168ffa'),
  'currency_salary': 'руб.',
  'max_salary': 200000,
  'min_salary': 90000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77436529?from=vacancy_search_list&query=%D0

  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b43d06070ad98169041'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 50000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77056849?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Ресечер / Менеджер по подбору персонала',
  'vacancy_salary': 'от 50\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b43d06070ad98169042'),
  'currency_salary': 'руб.',
  'max_salary': 38000,
  'min_salary': 35000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77494967?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер на пункт выдачи (Осиново)',
  'vacancy_salary': '35\u202f000 – 38\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b43d06070ad98169043'),
  'currency_salary': 'руб.',
  'max_salary': 80000,
  'min_salary': 44000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76953191?from=

...

 {'_id': ObjectId('64046b47d06070ad9816958a'),
  'currency_salary': 'руб.',
  'max_salary': 58000,
  'min_salary': 58000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77267844?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Заместитель управляющего магазином',
  'vacancy_salary': '58\u202f000 – 58\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b47d06070ad9816958b'),
  'currency_salary': 'руб.',
  'max_salary': 58000,
  'min_salary': 58000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77267982?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Заместитель управляющего магазином',
  'vacancy_salary': '58\u202f000 – 58\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b47d06070ad9816958c'),
  'currency_salary': 'руб.',
  'max_salary': None,
  'min_salary': 26500,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77135875?from=vacancy_search_list&quer

  'max_salary': 200000,
  'min_salary': 40000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76974559?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по оптовым продажам продуктов нефтепереработки',
  'vacancy_salary': '40\u202f000 – 200\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad981695cc'),
  'currency_salary': 'руб.',
  'max_salary': 30000,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76713231?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по аналитике, специалист, экономист, ассистент '
                  'руководителя',
  'vacancy_salary': 'до 30\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad981695cd'),
  'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77059202?from=vacancy_search_list&query=%D0%BC%D0%B

 {'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77337218?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по работе с ключевыми клиентами в канале СТО',
  'vacancy_salary': None,
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad98169613'),
  'currency_salary': 'руб.',
  'max_salary': 39000,
  'min_salary': 26000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77280208?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Помощник менеджера проектных продаж',
  'vacancy_salary': '26\u202f000 – 39\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad98169614'),
  'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76909203?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacanc

  'min_salary': 38000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77558890?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер-кассир',
  'vacancy_salary': 'от 38\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad98169646'),
  'currency_salary': None,
  'max_salary': None,
  'min_salary': None,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77323826?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по развитию',
  'vacancy_salary': None,
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b48d06070ad98169647'),
  'currency_salary': 'руб.',
  'max_salary': 57000,
  'min_salary': 40000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77085526?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Комплектовщик',
  'vacancy_salary': '40\u202f000 – 57\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {

  'vacancy_link': 'https://spb.hh.ru/vacancy/77199048?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер активных продаж',
  'vacancy_salary': 'от 48\u202f300 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b49d06070ad98169677'),
  'currency_salary': 'руб.',
  'max_salary': 120000,
  'min_salary': 30000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/77168156?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Менеджер по продажам в IT компанию (продажа программного '
                  'продукта)',
  'vacancy_salary': '30\u202f000 – 120\u202f000 руб.',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64046b49d06070ad98169678'),
  'currency_salary': 'руб.',
  'max_salary': 90000,
  'min_salary': 65000,
  'vacancy_link': 'https://spb.hh.ru/vacancy/76224124?from=vacancy_search_list&query=%D0%BC%D0%B5%D0%BD%D0%B5%D0%B4%D0%B6%D0%B5%D1%80',
  'vacancy_name': 'Торговый предст